In [12]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import re
import seaborn as sns
import nltk 
import heapq
from nltk import RegexpTokenizer as rpt
from nltk.corpus import stopwords as sw
from string import punctuation 

nltk.download('punkt')
nltk.download('stopwords')
stopwords = sw.words('portuguese')

data_url="./results.csv"
data = pd.read_csv(data_url).replace(np.nan, '', regex=True)
documents = data.text.count()
N = documents

def parse(text):
    words = []
    word_pattern = rpt(r'\w+')
    year_pattern = rpt(r'\d{4}')
    
    patterns = [word_pattern, year_pattern]
    
    for pattern in patterns:
        tokens = []
        for token in pattern.tokenize(text):
            if token not in stopwords and len(token) > 3:
                tokens.append(token)
        words.extend(tokens)
    return words


def build_index(dataset):
    document_index = 0
    index = {}
    M = len(dataset.text)
    for entry in dataset.text:
        document_index = document_index + 1
            
        for ngram in parse(entry):
            
            if ngram in index:
                if document_index in index[ngram]:
                    index[ngram][document_index] = index[ngram][document_index] + 1
                else:
                    index[ngram][document_index] = 1 
            else:
                index[ngram] = {document_index: 1}
                
    for i in index:
        k = len(index[i])
        index[i]["idf"] = math.log10((M + 1) / k)
    return index
                        
index = build_index(data)


[nltk_data] Downloading package punkt to /home/vinha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vinha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
def bin_query_vector(index, query):
    query_vector = []
    
    for word in index:
        if word in query.split():
            query_vector.append(True)
        else:
            query_vector.append(False)
            
    return query_vector

def bin_document_vector(index):
    document_vector = []
    
    for doc_id in range(1,documents+1):
        doc_vec = []
        for ngram in index:
            if doc_id in index[ngram].keys():
                doc_vec.append(True)
            else:
                doc_vec.append(False)
                
        document_vector.append(doc_vec)
    
    return document_vector
                

def f_bin(query_vector, doc_vector):
    rec = {}

    for doc_id in range(len(doc_vector)):
        sim = 0
        vector = doc_vector[doc_id]
        for i in range(len(vector)):
            sim += (query_vector[i] * vector[i])
        rec[doc_id+1] = sim
    
    return rec

def binary_vsm(index, query):
    query_vector = bin_query_vector(index, query)   
    doc_vector = bin_document_vector(index)
    
    return f_bin(query_vector, doc_vector)

In [14]:
def get_top5rank(score):
   
    df_tmp = pd.DataFrame(score.items(), columns=["document", "score"])
    df_tmp['r']= df_tmp.score.rank(ascending=False, method="first")
    df_tmp.sort_values("r", inplace = True)
    df_tmp = df_tmp[:5]
        
    return df_tmp

In [15]:
pd.concat([get_top5rank(binary_vsm(index, "lula")),
          get_top5rank(binary_vsm(index, "general")),
          get_top5rank(binary_vsm(index, "presidente"))]
          ,keys=["lula","general","presidente"])

document  score    r
lula       0          1      0  1.0
           1          2      0  2.0
           2          3      0  3.0
           3          4      0  4.0
           4          5      0  5.0
general    5          6      1  1.0
           18        19      1  2.0
           24        25      1  3.0
           62        63      1  4.0
           64        65      1  5.0
presidente 0          1      1  1.0
           1          2      1  2.0
           6          7      1  3.0
           7          8      1  4.0
           8          9      1  5.0

In [9]:
def tf_document_vector(index):
    document_vector = []
    
    for doc_id in range(1,documents+1):
        doc_vec = []
        for ngram in index:
            if doc_id in index[ngram].keys():
                y = index[ngram][doc_id]
                doc_vec.append(y)
            else:
                doc_vec.append(0)
                
        document_vector.append(doc_vec)
        
    return document_vector

def tf_query_vector(index, query):
    query_vector = []
    
    for ngram in index:
        w = 0
        for term in query.split():
            if ngram == term:
                w += 1
        query_vector.append(w)
        
    return query_vector

def f_tf(query_vector, doc_vector):
    rec = {}
    for doc_id in range(len(doc_vector)):
        sim = 0
        vector = doc_vector[doc_id]
        for i in range(len(vector)):
            sim += (query_vector[i] * vector[i])
        rec[doc_id+1] = sim
    return rec

def tf_vsm(index, query):
    query_vector = tf_query_vector(index, query)   
    doc_vector = tf_document_vector(index)
    
    return f_tf(query_vector, doc_vector)

pd.concat([get_top5rank(tf_vsm(index, "juíza")),
          get_top5rank(tf_vsm(index, "governo")),
          get_top5rank(tf_vsm(index, "seleção"))]
          ,keys=["juíza","governo","seleção"])

document  score    r
juíza   0           1      2  1.0
        1           2      1  2.0
        2           3      0  3.0
        3           4      0  4.0
        4           5      0  5.0
governo 172       173      7  1.0
        165       166      6  2.0
        83         84      5  3.0
        209       210      5  4.0
        24         25      4  5.0
seleção 114       115      4  1.0
        115       116      3  2.0
        118       119      3  3.0
        120       121      3  4.0
        56         57      2  5.0

In [10]:
def tfidf_document_vector(index):
    document_vector = []
    
    for doc_id in range(1,documents+1):
        doc_vec = []
        for ngram in index:
            if doc_id in index[ngram].keys():
                y = index[ngram][doc_id] * index[ngram]['idf']
                doc_vec.append(y)
            else:
                doc_vec.append(0)
                
        document_vector.append(doc_vec)
        
    return document_vector


def tfidf_vsm(index, query):
    query_vector = tf_query_vector(index, query)   
    doc_vector = tfidf_document_vector(index)
    
    return f_tf(query_vector, doc_vector)

pd.concat([get_top5rank(tfidf_vsm(index, "juíza")),
          get_top5rank(tfidf_vsm(index, "governo")),
          get_top5rank(tfidf_vsm(index, "seleção"))]
          ,keys=["juíza","governo","seleção"])

document     score    r
juíza   0           1  4.193820  1.0
        1           2  2.096910  2.0
        2           3  0.000000  3.0
        3           4  0.000000  4.0
        4           5  0.000000  5.0
governo 172       173  6.548794  1.0
        165       166  5.613252  2.0
        83         84  4.677710  3.0
        209       210  4.677710  4.0
        24         25  3.742168  5.0
seleção 114       115  4.669964  1.0
        115       116  3.502473  2.0
        118       119  3.502473  3.0
        120       121  3.502473  4.0
        56         57  2.334982  5.0

In [11]:
def f_bm25(query_vector, doc_vector, k):
    rec = {}
    for doc_id in range(len(doc_vector)):
        sim = 0
        vector = doc_vector[doc_id]
        for i in range(len(vector)):
            if vector[i] != 0:
                y = (k+1) * query_vector[i]
                dom = (query_vector[i] * y)/(query_vector[i]+k)
                sim += (dom * math.log10((documents + 1)/vector[i]))
        rec[doc_id+1] = sim
    return rec
    

def bm25_vsm(index, query, k):
    query_vector = tf_query_vector(index, query)   
    doc_vector = tf_document_vector(index)
    
    return f_bm25(query_vector, doc_vector, k)

pd.concat([get_top5rank(bm25_vsm(index, "juíza", 100)),
          get_top5rank(bm25_vsm(index, "governo", 100)),
          get_top5rank(bm25_vsm(index, "seleção", 100))]
          ,keys=["juíza","governo","seleção"])

document    score    r
juíza   1          2  2.39794  1.0
        0          1  2.09691  2.0
        2          3  0.00000  3.0
        3          4  0.00000  4.0
        4          5  0.00000  5.0
governo 2          3  2.39794  1.0
        20        21  2.39794  2.0
        41        42  2.39794  3.0
        72        73  2.39794  4.0
        94        95  2.39794  5.0
seleção 40        41  2.39794  1.0
        48        49  2.39794  2.0
        54        55  2.39794  3.0
        57        58  2.39794  4.0
        61        62  2.39794  5.0